# Tutorial (5): how to define Hamiltonian and ansatz
This example shows how one can set a user-defined Hamiltonian and user-defined ansatz (for VQE).  
Quket takes a full advantage of OpenFermion library, and these user-defined objects should be in the OpenFermion format.

In [1]:
# import necessary modules
from openfermion import *
from quket import *
from quket.utils import *
import quket.config as cf
import numpy as np

mpi4py is not imported. no MPI.


[(Example 1) Hubbard Hamiltonian](#(Example-1)-Hubbard-Hamiltonian)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[(1.1) Define Hamiltonian in Quket using option `hamiltonian`](#(1.1)-Define-Hamiltonian-in-Quket-using-option-hamiltonian)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[(1.2) Run `fci2qubit()` to determine the ground state](#(1.2)-Run-fci2qubit()-to-determine-the-ground-state)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[(1.3) Run `vqe()`](#(1.3)-Run-vqe())  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[(1.4) Transform the state to Jordan-Wigner's Particle-Hole representation](#(1.4)-Transform-the-state-to-Jordan-Wigner's-Particle-Hole-representation)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[(1.5) Tapering-off qubits](#(1.5)-Tapering-off-qubits)

[(Example2) Modified Kitaev Hamiltonian](#(Example2)-Modified-Kitaev-Hamiltonian)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[(2.1) Define the modified Kitaev Hamiltonian](#(2.1)-Define-the-modified-Kitaev-Hamiltonian)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[(2.2) Run `fci2qubit()` to determine the ground state](#(2.2)-Run-fci2qubit()-to-determine-the-ground-state)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[(2.3) Manually define `pauli_list`](#(2.3)-Manually-define-pauli_list)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[(2.4) Run `vqe()`](#(2.4)-Run-vqe())


## (Example 1) Hubbard Hamiltonian

 Define the Hubbard Hamiltonian using creation and annihilation operator

In [2]:
### Hubbard model as FermionOperator (OpenFermion part)
nx = 4
ny = 1
t = 1
U = 4
hubbard = fermi_hubbard(nx, ny, t, U,periodic=False)

### (1.1) Define Hamiltonian in Quket using option `hamiltonian`  
Here, we use the following options:
`ansatz`: Hamiltonian variational ansatz (HVA)  
`mapping`: Bravyi-Kitaev mapping  
`layers`: 2 layers of HVA  
`theta_guess`: Randomized VQE parameter guess  
`det`: Initial state is `|00000011>` (in the Jordan-Wigner representation: this will be transfromed to the Bravyi-Kitaev representation `|00000001>`)  
`maxiter`: Maximum iteration of VQE

In [3]:
Hub = create(hamiltonian=hubbard, 
             ansatz='hva', 
             mapping='bk', 
             layers=2, 
             theta_guess='random', 
             det='|0011>',
             maxiter=200)

Simulation detail
Hamiltonian:
(4+0j) [] +
(-0.5+0j) [X0 Y1 Y2] +
(0.5+0j) [Y0 Y1 X2] +
(-1+0j) [Z0] +
(0.5+0j) [Z0 X1 Z3] +
(-1+0j) [Z0 Z1] +
(-0.5+0j) [X1 Z2] +
(1+0j) [Z1] +
(-0.5+0j) [Z1 X2 Y3 Y4 X5] +
(0.5+0j) [Z1 Y2 Y3 X4 X5] +
(-0.5+0j) [Z1 Z2 Y3 Y5] +
(-1+0j) [Z1 Z2 Z3] +
(1+0j) [Z1 Z3] +
(-1+0j) [Z2] +
(-0.5+0j) [X3 Z4 X5] +
(0.5+0j) [Z3 Z4 X5 Z7] +
(-1+0j) [Z3 Z5 Z6 Z7] +
(1+0j) [Z3 Z5 Z7] +
(-0.5+0j) [X4 Y5 Y6] +
(0.5+0j) [Y4 Y5 X6] +
(-1+0j) [Z4] +
(-1+0j) [Z4 Z5] +
(-0.5+0j) [X5 Z6] +
(1+0j) [Z5] +
(-1+0j) [Z6]
Initial state:
    Basis            Coef
| 00000001 > : +1.0000 +0.0000i

Sequential Tapering-Off Results:
List of redundant qubits:  [0, 7]
Qubit: 0    Tau: 1.0 [Z0 Z2 Z4 Z6]
Qubit: 7    Tau: 1.0 [Z7]



### (1.2) Run `fci2qubit()` to determine the ground state

In [4]:
Hub.fci2qubit()

Davidson convergence achieved.
FCI in Qubits
(FCI state : E = -2.6249422715)
    Basis            Coef
| 00000011 > : +0.2074 +0.0000i
| 00000100 > : +0.1734 +0.0000i
| 00000110 > : -0.2074 +0.0000i
| 00001011 > : +0.2349 +0.0000i
| 00001100 > : +0.3083 +0.0000i
| 00010000 > : +0.1734 +0.0000i
| 00101011 > : +0.3083 +0.0000i
| 00101100 > : +0.3670 +0.0000i
| 00110000 > : +0.2074 +0.0000i
| 00111000 > : -0.3670 +0.0000i
| 00111010 > : -0.3083 +0.0000i
| 01001000 > : -0.3083 +0.0000i
| 01001010 > : -0.2349 +0.0000i
| 01100000 > : -0.2074 +0.0000i



### (1.3) Run `vqe()` 

In [5]:
Hub.vqe()

      1: E[hva] = +0.928602316235  <S**2> = +0.000000  Fidelity = 0.041020  Grad = 6.50e+00  CPU Time =    0.09735  (0.00 / step)
      2: E[hva] = +0.928403555616  <S**2> = +0.000000  Fidelity = 0.040899  Grad = 6.53e+00  CPU Time =    0.02976  (0.00 / step)
      3: E[hva] = -0.179022188671  <S**2> = +0.000000  Fidelity = 0.024858  Grad = 3.54e+00  CPU Time =    0.01539  (0.00 / step)
      4: E[hva] = -0.459292593423  <S**2> = +0.000000  Fidelity = 0.038665  Grad = 3.17e+00  CPU Time =    0.03038  (0.00 / step)
      5: E[hva] = -0.713141870433  <S**2> = +0.000000  Fidelity = 0.067977  Grad = 2.85e+00  CPU Time =    0.01606  (0.00 / step)
      6: E[hva] = -1.047971689174  <S**2> = +0.000000  Fidelity = 0.132295  Grad = 3.15e+00  CPU Time =    0.03009  (0.00 / step)
      7: E[hva] = -1.265403026190  <S**2> = +0.000000  Fidelity = 0.243598  Grad = 3.41e+00  CPU Time =    0.01563  (0.00 / step)
      8: E[hva] = -1.617558103624  <S**2> = +0.000000  Fidelity = 0.324038  Grad = 1.23e+0

### (1.4) Transform the state to Jordan-Wigner's Particle-Hole representation
For this purpose, `transform_state_bk2jw()` can be used.  
The opposite transformation is done with `transform_state_jw2bk()`.

In [6]:
jw_state = transform_state_bk2jw(Hub.state)
print_state(jw_state)

    Basis            Coef
| 00000110 > : +0.1556 -0.1370i
| 00001001 > : -0.1558 +0.1370i
| 00001100 > : -0.1304 +0.1143i
| 00010010 > : +0.2285 -0.2011i
| 00011000 > : +0.2779 -0.2430i
| 00100001 > : -0.2312 +0.2014i
| 00100100 > : -0.2771 +0.2432i
| 00110000 > : -0.1308 +0.1148i
| 01000010 > : +0.1803 -0.1571i
| 01001000 > : +0.2292 -0.2015i
| 01100000 > : +0.1548 -0.1361i
| 10000001 > : -0.1780 +0.1575i
| 10000100 > : -0.2316 +0.2021i
| 10010000 > : -0.1565 +0.1368i



Transform back to the bravyi-kitaev representation using `transform_state_jw2bk()`:

In [7]:
bk_state = transform_state_jw2bk(jw_state)
print_state(bk_state)
print("They are the same state:", bk_state == Hub.state)

    Basis            Coef
| 00000011 > : -0.1558 +0.1370i
| 00000100 > : -0.1304 +0.1143i
| 00000110 > : +0.1556 -0.1370i
| 00001011 > : -0.1780 +0.1575i
| 00001100 > : -0.2316 +0.2021i
| 00010000 > : -0.1308 +0.1148i
| 00101011 > : -0.2312 +0.2014i
| 00101100 > : -0.2771 +0.2432i
| 00110000 > : -0.1565 +0.1368i
| 00111000 > : +0.2779 -0.2430i
| 00111010 > : +0.2285 -0.2011i
| 01001000 > : +0.2292 -0.2015i
| 01001010 > : +0.1803 -0.1571i
| 01100000 > : +0.1548 -0.1361i

They are the same state: True


### (1.5) Tapering-off qubits
Symmetry-search and tapering-off qubits can be performed for user-defined Hamiltonians.

In [8]:
Hub.taper_off()

States     transformed.
Operators  transformed.
pauli_list transformed.
theta_list transformed.


In [9]:
print_state(Hub.state)

   Basis           Coef
| 000001 > : -0.1558 +0.1370i
| 000010 > : +0.1304 -0.1143i
| 000011 > : -0.1556 +0.1370i
| 000101 > : -0.1780 +0.1575i
| 000110 > : +0.2316 -0.2021i
| 001000 > : +0.1308 -0.1148i
| 010101 > : -0.2312 +0.2014i
| 010110 > : +0.2771 -0.2432i
| 011000 > : +0.1565 -0.1368i
| 011100 > : -0.2779 +0.2430i
| 011101 > : -0.2285 +0.2011i
| 100100 > : -0.2292 +0.2015i
| 100101 > : -0.1803 +0.1571i
| 110000 > : -0.1548 +0.1361i



In [10]:
print("Qubits in the tapered-mapping", Hub.n_qubits)
print("Qubits in the original mapping", Hub._n_qubits)

Qubits in the tapered-mapping 6
Qubits in the original mapping 8


In [11]:
Hub.vqe()

      1: E[hva] = -2.624651318146  <S**2> = +0.000000  Fidelity = 0.999931  Grad = 1.58e-03  CPU Time =    0.04231  (0.00 / step)
      2: E[hva] = -2.624651462452  <S**2> = +0.000000  Fidelity = 0.999931  Grad = 1.38e-03  CPU Time =    0.01476  (0.00 / step)
      3: E[hva] = -2.624652019083  <S**2> = +0.000000  Fidelity = 0.999931  Grad = 1.56e-03  CPU Time =    0.01620  (0.00 / step)
      4: E[hva] = -2.624652561294  <S**2> = +0.000000  Fidelity = 0.999930  Grad = 2.14e-03  CPU Time =    0.01568  (0.00 / step)
      5: E[hva] = -2.624653254218  <S**2> = +0.000000  Fidelity = 0.999929  Grad = 2.39e-03  CPU Time =    0.01566  (0.00 / step)
      6: E[hva] = -2.624654043103  <S**2> = +0.000000  Fidelity = 0.999929  Grad = 2.12e-03  CPU Time =    0.01546  (0.00 / step)
      7: E[hva] = -2.624655560025  <S**2> = +0.000000  Fidelity = 0.999930  Grad = 2.98e-03  CPU Time =    0.01607  (0.00 / step)
      8: E[hva] = -2.624657114245  <S**2> = +0.000000  Fidelity = 0.999932  Grad = 5.43e-0

## (Example2) Modified Kitaev Hamiltonian
This sample performs VQE with a modified Kitaev Hamiltonian and Hamiltonian Variational Ansatz of arXiv:2108.13375.

```               
  0           3 
  Z \   X   / Z
      1 - 2  
    Y |   |  Y
      6 - 4  
    /   X   \
  7 Z      Z  5 
                

  Eg = -4.0100  (TCz in Table I)
  Eg = -4.2476  (TCz+h in Table I)
```



### (2.1) Define the modified Kitaev Hamiltonian

In [12]:
### Define the modified Kitaev Hamiltonian ###
### Parameters for TCz+h
Jx = -0.1
Jy = Jx
Jz = -1
hx = 0.05/np.sqrt(3)
hy = hx 
hz = hx 

### User-defined Hamiltonian in the OpenFermion format.
hamiltonian =f"{Jx} [X1 X2] + {Jx} [X4 X6] \
             + {Jy} [Y1 Y6] + {Jy} [Y2 Y4] \
             + {Jz} [Z0 Z1] + {Jz} [Z2 Z3] + {Jz} [Z4 Z5] + {Jz} [Z6 Z7] \
             + {hx} [X0] + {hx} [X1] + {hx} [X2] + {hx} [X3] \
             + {hx} [X4] + {hx} [X5] + {hx} [X6] + {hx} [X7] \
             + {hy} [Y0] + {hy} [Y1] + {hy} [Y2] + {hy} [Y3] \
             + {hy} [Y4] + {hy} [Y5] + {hy} [Y6] + {hy} [Y7] \
             + {hz} [Z0] + {hz} [Z1] + {hz} [Z2] + {hz} [Z3] \
             + {hz} [Z4] + {hz} [Z5] + {hz} [Z6] + {hz} [Z7] "

In [13]:
### Either of the followings works
BK = create(hamiltonian=QubitOperator(hamiltonian))
#BK = create(hamiltonian=hamiltonian)

Initial state is not supplied, so we use |0>.
You may specify the initial state by one of the following ways:

  Q.set(det='|0011>')
for product state,
  Q.set(det='0.5 * |0011> + 0.5 * |0101>')
for entangled state, and
  Q.init_state = QuantumState(Q.n_qubits)
  ...
to directly subsititute user-defined QuantumState to init_state.

Simulation detail
Hamiltonian:
0.02886751345948129 [X0] +
0.02886751345948129 [Y0] +
0.02886751345948129 [Z0] +
-1.0 [Z0 Z1] +
0.02886751345948129 [X1] +
-0.1 [X1 X2] +
0.02886751345948129 [Y1] +
-0.1 [Y1 Y6] +
0.02886751345948129 [Z1] +
0.02886751345948129 [X2] +
0.02886751345948129 [Y2] +
-0.1 [Y2 Y4] +
0.02886751345948129 [Z2] +
-1.0 [Z2 Z3] +
0.02886751345948129 [X3] +
0.02886751345948129 [Y3] +
0.02886751345948129 [Z3] +
0.02886751345948129 [X4] +
-0.1 [X4 X6] +
0.02886751345948129 [Y4] +
0.02886751345948129 [Z4] +
-1.0 [Z4 Z5] +
0.02886751345948129 [X5] +
0.02886751345948129 [Y5] +
0.02886751345948129 [Z5] +
0.02886751345948129 [X6] +
0.028867513459481

### (2.2) Run `fci2qubit()` to determine the ground state

In [14]:
BK.fci2qubit()

Davidson convergence achieved.
FCI in Qubits
(FCI state : E = -4.2475747377)
    Basis            Coef
| 11111111 > : +0.9965 -0.0000i



### (2.3) Manually define `pauli_list`
If only `hamiltonian` is defined, `Quket` complains that there is no variational circuit.  
We can provide either `ansatz` or `pauli_list` to construct a variational circuit.  
Here, let us define `pauli_list` manually, which can be done by using option `pauli_list`.  
  
In a `QuketData` instance, the attribute `QuketData.pauli_list` is a list of `QubitOperator`s.  
When specifying `pauli_list` as an option of `set()` or `create()`, it can be either `QubitOperator` or `str` (which will be converted).

In [15]:
# As an example, we take HVA but define it manually.
pauli_list =['[X1 X2]', '[X4 X6]', '[Y1 Y6]', '[Y2 Y4]', 
             '[Z0 Z1]', '[Z2 Z3]', '[Z4 Z5]', '[Z6 Z7]', 
             '[X0]', '[X1]', '[X2]', '[X3]',
             '[X4]', '[X5]', '[X6]', '[X7]',
             '[Y0]', '[Y1]', '[Y2]', '[Y3]',
             '[Y4]', '[Y5]', '[Y6]', '[Y7]',
             '[Z0]', '[Z1]', '[Z2]', '[Z3]',
             '[Z4]', '[Z5]', '[Z6]', '[Z7]',]
# Additionally let us set other options.
BK.set(pauli_list=pauli_list, layers=1, theta_guess="random", det="|11111111>")
BK.initialize()

Simulation detail
Hamiltonian:
0.02886751345948129 [X0] +
0.02886751345948129 [Y0] +
0.02886751345948129 [Z0] +
-1.0 [Z0 Z1] +
0.02886751345948129 [X1] +
-0.1 [X1 X2] +
0.02886751345948129 [Y1] +
-0.1 [Y1 Y6] +
0.02886751345948129 [Z1] +
0.02886751345948129 [X2] +
0.02886751345948129 [Y2] +
-0.1 [Y2 Y4] +
0.02886751345948129 [Z2] +
-1.0 [Z2 Z3] +
0.02886751345948129 [X3] +
0.02886751345948129 [Y3] +
0.02886751345948129 [Z3] +
0.02886751345948129 [X4] +
-0.1 [X4 X6] +
0.02886751345948129 [Y4] +
0.02886751345948129 [Z4] +
-1.0 [Z4 Z5] +
0.02886751345948129 [X5] +
0.02886751345948129 [Y5] +
0.02886751345948129 [Z5] +
0.02886751345948129 [X6] +
0.02886751345948129 [Y6] +
0.02886751345948129 [Z6] +
-1.0 [Z6 Z7] +
0.02886751345948129 [X7] +
0.02886751345948129 [Y7] +
0.02886751345948129 [Z7]
Initial state:
    Basis            Coef
| 11111111 > : +1.0000 +0.0000i

user-defined pauli_list found.
Sequential Tapering-Off Results:
No symmetry found.



#### Check `pauli_list` attribute.

In [16]:
BK.pauli_list

[1.0 [X1 X2],
 1.0 [X4 X6],
 1.0 [Y1 Y6],
 1.0 [Y2 Y4],
 1.0 [Z0 Z1],
 1.0 [Z2 Z3],
 1.0 [Z4 Z5],
 1.0 [Z6 Z7],
 1.0 [X0],
 1.0 [X1],
 1.0 [X2],
 1.0 [X3],
 1.0 [X4],
 1.0 [X5],
 1.0 [X6],
 1.0 [X7],
 1.0 [Y0],
 1.0 [Y1],
 1.0 [Y2],
 1.0 [Y3],
 1.0 [Y4],
 1.0 [Y5],
 1.0 [Y6],
 1.0 [Y7],
 1.0 [Z0],
 1.0 [Z1],
 1.0 [Z2],
 1.0 [Z3],
 1.0 [Z4],
 1.0 [Z5],
 1.0 [Z6],
 1.0 [Z7]]

### (2.4) Run `vqe()`

In [17]:
BK.vqe()

      1: E[user-defined] = -4.237670453615  <S**2> = +0.000000  Fidelity = 0.996664  Grad = 2.75e-02  CPU Time =    0.03969  (0.00 / step)
      2: E[user-defined] = -4.237764189519  <S**2> = +0.000000  Fidelity = 0.996735  Grad = 8.37e-03  CPU Time =    0.01241  (0.00 / step)
      3: E[user-defined] = -4.237765987538  <S**2> = +0.000000  Fidelity = 0.996737  Grad = 6.14e-03  CPU Time =    0.01129  (0.00 / step)
      4: E[user-defined] = -4.237768397009  <S**2> = +0.000000  Fidelity = 0.996736  Grad = 7.88e-05  CPU Time =    0.01172  (0.00 / step)
      5: E[user-defined] = -4.237768460613  <S**2> = +0.000000  Fidelity = 0.996736  Grad = 6.65e-04  CPU Time =    0.03857  (0.00 / step)
      6: E[user-defined] = -4.237780674350  <S**2> = +0.000000  Fidelity = 0.996617  Grad = 5.96e-02  CPU Time =    0.03932  (0.00 / step)
      7: E[user-defined] = -4.237780976663  <S**2> = +0.000000  Fidelity = 0.996599  Grad = 6.44e-02  CPU Time =    0.03095  (0.00 / step)
      8: E[user-defined] = 

#### There does not seem to be any symmetry in the modified Kitaev Hamiltonian, 
#### and hence `taper_off()` does nothing.

In [18]:
BK.taper_off()

No qubits to be tapered-off. No transformation done
